In [1]:
import pandas as pd
import requests as req
import bs4
import xlsxwriter
import time

In [2]:
file_name = 'journal_links_db.xlsx'
xlsx = pd.ExcelFile(file_name)
sheet_names = xlsx.sheet_names
# print(sheet_names)
# ['mk', 'ct', 'br', 'gu', 'er', 'bq', 'sq', 'pr', 'nm', 'ch']

In [3]:
def get_abstract(soup):
    try:
        abstract_header = soup.find(class_='sectionInfo abstractSectionHeading')
        abstract = abstract_header.find_next_sibling('p')
        return abstract.get_text()
    except:
        print('err')
        return 'err'

def get_authors(soup):
    try:
        names = ""
        authors_class = soup.find(class_='rlist--inline loa hidden-xs loa-accordion')
        authors = authors_class.findChildren("a")
        for author in authors:
            name = author.get_text()
            if name:
                #add names to list and remove whitespace
                names += (name.strip()) + ","
        print(names)
        return names[:-1] # remove last comma 
    except:
        print('err')
        return 'err'

def get_title(soup):
    try:
        title = soup.find(class_='citation__title')
        print(title)
        return title.get_text()
    except:
        print('err')
        return 'err'

In [4]:
# create workbook for the database
# if creating new one remove old or name otherwise e.g. "database2.xlsx"
workbook = xlsxwriter.Workbook("database.xlsx")

In [5]:
# script to write all journal info to database
# different categories will be written on different sheets named 'mk', 'ct', ...
for category in sheet_names:
    print("CATEGORY############################################" + category)
    # some pandas-excel-xlsx magic here... ¯\_(ツ)_/¯
    df = pd.read_excel(file_name, sheet_name=category)
    worksheet = workbook.add_worksheet(category)
    worksheet.write(0, 0, 'title')
    worksheet.write(0, 1, 'authors')
    worksheet.write(0, 2, 'abstract')
    for i in range(0, 9): # reading links from pandas_tables starts at index 0, if want to run all change range to (0,99)
        link = df.iloc[i].values[0]
        print(str(i) + " " + link)
        # get article
        response = req.get(link)
        article_soup = bs4.BeautifulSoup(response.content, 'html.parser')
        # we have to increment i by 0 so we can have headers in index 0
        worksheet.write(i + 1, 0, get_title(article_soup))
        worksheet.write(i + 1, 1, get_authors(article_soup))
        worksheet.write(i + 1, 2, get_abstract(article_soup))
        time.sleep(30)
workbook.close()

CATEGORY############################################mk
0 https://doi.org/10.1148/radiol.211818
<h1 class="citation__title"><a href="/doi/full/10.1148/radiol.211818" title="Dual-Energy CT for Detecting Painful Knee Prosthesis Loosening">Dual-Energy CT for Detecting Painful Knee Prosthesis Loosening</a></h1>
Giovanni Foti,Chiara Longo,Mirko D’Onofrio,Simone Natali,Gianluca Piovan,Eugenio Oliboni,Venanzio Iacono,Massimo Guerriero,Claudio Zorzi,


KeyboardInterrupt: 